# Hybrid Banking Recommendation System: ALS + LLM
## Zenith Bank Product Recommendation Engine

This notebook implements a hybrid recommendation system that combines:
1. **Collaborative Filtering (ALS)** - For data-driven product recommendations
2. **LLM (GPT-4)** - For context awareness, explanations, and personalization

**Architecture:**
- ALS generates base recommendations from transaction patterns
- LLM enhances with context, explanations, and customer segmentation
- Feature engineering creates rich customer profiles

## Cell 1: Install Required Libraries

In [1]:
# Install required libraries
#!pip install -q openai python-dotenv implicit scikit-learn scipy pyspark

## Cell 2: Import Libraries

In [2]:
import os
import json
import warnings
from datetime import datetime, timedelta
from typing import List, Dict, Any, Optional, Tuple

import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, ndcg_score

from pyspark.sql import SparkSession
import pandas as pd
import backoff


import time
import openai
from openai import OpenAI
from dotenv import load_dotenv
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql import Window
import json
from decimal import Decimal

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Configure
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ All libraries imported successfully")

✅ All libraries imported successfully


## Cell 3: Configuration and Setup

In [3]:
# ============================================================================
# API KEYS AND CONFIGURATION (loaded from .env)
# ============================================================================
from dotenv import load_dotenv
import os
import json

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

CONFIG = {
    'als': {
        'factors': int(os.getenv('ALS_FACTORS', 50)),
        'regularization': float(os.getenv('ALS_REGULARIZATION', 0.01)),
        'iterations': int(os.getenv('ALS_ITERATIONS', 15)),
        'alpha': float(os.getenv('ALS_ALPHA', 40.0))
    },
    'llm': {
        'model': os.getenv('LLM_MODEL', 'gpt-4o-mini'),
        'temperature': float(os.getenv('LLM_TEMPERATURE', 0.3)),
        'max_tokens': int(os.getenv('LLM_MAX_TOKENS', 2000))
    },
    'recommendation': {
        'top_n': int(os.getenv('RECOMMENDATION_TOP_N', 5)),
        'final_n': int(os.getenv('RECOMMENDATION_FINAL_N', 3))
    },
    'feature_engineering': {
        'recency_days': int(os.getenv('FEATURE_RECENCY_DAYS', 90)),
        'min_transactions': int(os.getenv('FEATURE_MIN_TRANSACTIONS', 3))
    }
}

print("\n📊 Configuration Loaded from .env:")
print(json.dumps(CONFIG, indent=2))



📊 Configuration Loaded from .env:
{
  "als": {
    "factors": 50,
    "regularization": 0.01,
    "iterations": 15,
    "alpha": 40.0
  },
  "llm": {
    "model": "gpt-4o-mini",
    "temperature": 0.3,
    "max_tokens": 2000
  },
  "recommendation": {
    "top_n": 5,
    "final_n": 3
  },
  "feature_engineering": {
    "recency_days": 90,
    "min_transactions": 3
  }
}


## Cell 4: Load Data

In [4]:

# Initialize Spark
spark = SparkSession.builder \
    .appName("PinnacleAI_DataLoad") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

print("✅ Spark session initialized successfully")

# File paths
TRANSACTIONS_TABLE = r'C:\Users\adeye\Documents\Pinnacle-AI\dataset\transaction.csv'
PRODUCTS_TABLE = r'C:\Users\adeye\Documents\Pinnacle-AI\dataset\product.csv'
CONVERSATIONS_TABLE = r'C:\Users\adeye\Documents\Pinnacle-AI\dataset\interaction_pd.csv'
CUSTOMERS_TABLE = r'C:\Users\adeye\Documents\Pinnacle-AI\dataset\customers.csv'

print("📂 Loading datasets...\n")

try:
    df_transactions = spark.read.option("header", True).csv(TRANSACTIONS_TABLE)
    df_products = spark.read.option("header", True).csv(PRODUCTS_TABLE)
    df_conversations = spark.read.option("header", True).csv(CONVERSATIONS_TABLE)
    df_customers = spark.read.option("header", True).csv(CUSTOMERS_TABLE)

    print(f"✅ Loaded {df_transactions.count():,} transactions")
    print(f"✅ Loaded {df_products.count():,} products")
    print(f"✅ Loaded {df_conversations.count():,} conversations")
    print(f"✅ Loaded {df_customers.count():,} customers")

    print("\n📊 Schema Preview:")
    df_transactions.printSchema()

except Exception as e:
    print(f"❌ Error: {e}")


✅ Spark session initialized successfully
📂 Loading datasets...

✅ Loaded 1,200,000 transactions
✅ Loaded 42 products
✅ Loaded 1,200,000 transactions
✅ Loaded 42 products
✅ Loaded 3,164 conversations
✅ Loaded 3,164 conversations
✅ Loaded 100,000 customers

📊 Schema Preview:
root
 |-- Customer_ID: string (nullable = true)
 |-- Trans_Amount: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Destination: string (nullable = true)
 |-- Deb_or_credit: string (nullable = true)
 |-- Narration: string (nullable = true)
 |-- Tran_Id: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Account_Type: string (nullable = true)

✅ Loaded 100,000 customers

📊 Schema Preview:
root
 |-- Customer_ID: string (nullable = true)
 |-- Trans_Amount: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Destination: string (nullable = true)
 |-- Deb_or_credit: string (nullable = true)
 |-- Narration: string (nullable = true)
 |-- Tran_Id: string (nullable = true)
 |-

## Cell 5: Data Exploration

In [5]:
print("\n TRANSACTIONS SAMPLE:")
df_transactions.show(5, truncate=False)

print("\n PRODUCTS SAMPLE:")
df_products.show(5, truncate=False)

print("\n CUSTOMERS SAMPLE:")
df_customers.show(5, truncate=False)

print("\n CONVERSATIONS SAMPLE:")
df_conversations.show(5, truncate=False)

print("\n TRANSACTION STATISTICS:")
df_transactions.describe().show(truncate=False)

print("\n Missing Values Per Column:")
missing_df = df_transactions.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c)
    for c in df_transactions.columns
])
missing_df.show(truncate=False)

print("\n DATA OVERVIEW:")
print(f"Total Rows: {df_transactions.count():,}")
for col in df_transactions.columns:
    unique_count = df_transactions.select(col).distinct().count()
    print(f"Unique {col}: {unique_count:,}")



 TRANSACTIONS SAMPLE:
+-----------+------------+----------+---------------------+-------------+--------------------+-----------+---------+----------------------------+
|Customer_ID|Trans_Amount|Date      |Destination          |Deb_or_credit|Narration           |Tran_Id    |Category |Account_Type                |
+-----------+------------+----------+---------------------+-------------+--------------------+-----------+---------+----------------------------+
|ZB060544   |2161947.29  |2023-01-01|Maintenance Services |D            |service charge      |TR000054680|Housing  |Timeless Account Savings    |
|ZB032114   |11331.88    |2023-01-01|Mobil Filling Station|D            |premium motor spirit|TR000874920|Transport|Gold Premium Current Account|
|ZB096997   |497269.45   |2023-01-01|JAMB Registration    |D            |school fees payment |TR000315856|Education|Timeless Account Current    |
|ZB011060   |382993.75   |2023-01-01|Slot Limited         |D            |fashion items       |TR00082

## Cell 6: Feature Engineering - Create Interaction Matrix

In [10]:
from openai import OpenAI
import time
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql import Window
import json
from decimal import Decimal

def create_customer_product_interactions(df_custs, df_products, df_trans,
                                          openai_api_key=None,
                                          model_name="gpt-4o-mini",
                                          customer_sample_size=None,
                                          batch_size=30,
                                          temperature=0.3,
                                          top_n_products=10,
                                          rate_limit_delay=0.5,
                                          additional_rules=None,
                                          use_transaction_data=True,
                                          transaction_weight=0.7):
    """
    HYBRID: Combines OpenAI intelligent matching with real transaction data
    Enhanced with transaction description analysis
    All parameters configurable, no hard-coded values
    Uses Spark for distributed processing
   
    Args:
        df_custs: Customer DataFrame with demographics and Account_Type
        df_products: Product catalog DataFrame
        df_trans: Transaction DataFrame (with Description field)
        openai_api_key: OpenAI API key (required)
        model_name: OpenAI model to use
        customer_sample_size: Number of customers to process (None = all)
        batch_size: Number of customers per API call
        temperature: LLM temperature setting
        top_n_products: Number of products to recommend per customer
        rate_limit_delay: Delay between API calls in seconds
        additional_rules: Optional list of custom business rules
        use_transaction_data: Whether to include transaction data
        transaction_weight: Weight for transaction scores (0-1)
    """
    start_time = time.time()
    print("🔨 Creating interaction matrix (Hybrid: OpenAI + Transaction Descriptions)...\n")
   
    # Validate inputs
    if openai_api_key is None:
        raise ValueError("openai_api_key parameter is required")
   
    # Initialize OpenAI client
    client = OpenAI(api_key=openai_api_key)
   
    # Get all product names
    all_product_names = [row.Product_Name for row in df_products.select('Product_Name').collect()]
    print(f"   Products in catalog: {len(all_product_names)}")
   
    # =========================================================================
    # 1. IDENTIFY CURRENT PRODUCTS FROM ACCOUNT_TYPE (SPARK)
    # =========================================================================
    print("\n   → Identifying current products from Account_Type...")
   
    df_custs_with_keywords = df_custs.withColumn(
        'account_keywords',
        F.regexp_extract(F.lower(F.col('Account_Type')),
                        r'(current|savings|sme|student|premium|platinum|children|aspire)', 1)
    )
   
    df_products_keywords = df_products.withColumn(
        'product_keywords',
        F.regexp_extract(F.lower(F.col('Product_Name')),
                        r'(current|savings|sme|student|premium|platinum|children|aspire)', 1)
    )
   
    keyword_map = df_products_keywords.filter(F.col('product_keywords') != '') \
        .select('product_keywords', 'Product_Name') \
        .distinct()
   
    df_custs_with_current = df_custs_with_keywords.join(
        keyword_map,
        df_custs_with_keywords.account_keywords == keyword_map.product_keywords,
        'left'
    ).withColumnRenamed('Product_Name', 'current_product')
   
    customer_current_products_df = df_custs_with_current.filter(
        F.col('current_product').isNotNull()
    ).groupBy('Customer_ID').agg(
        F.collect_set('current_product').alias('current_products')
    )
   
    current_products_count = customer_current_products_df.count()
    print(f"      Found current products for {current_products_count:,} customers")
   
    customer_current_products = {
        row.Customer_ID: set(row.current_products)
        for row in customer_current_products_df.collect()
    }
   
    # =========================================================================
    # 2. PREPARE CUSTOMER PROFILES (SPARK)
    # =========================================================================
    print("\n   → Preparing customer profiles...")
   
    customer_profiles = df_custs.select([c for c in df_custs.columns])
    profiles_count = customer_profiles.count()
    print(f"      Total customers: {profiles_count:,}")
   
    # =========================================================================
    # 3. SAMPLE CUSTOMERS IF SPECIFIED
    # =========================================================================
    if customer_sample_size is not None and profiles_count > customer_sample_size:
        print(f"      Sampling {customer_sample_size:,} customers from {profiles_count:,} total")
        customer_profiles_sample = customer_profiles.sample(
            fraction=customer_sample_size/profiles_count,
            seed=42
        ).limit(customer_sample_size)
    else:
        print(f"      Processing all {profiles_count:,} customers")
        customer_profiles_sample = customer_profiles
   
    # =========================================================================
    # 4. PREPARE PRODUCT CATALOG WITH RULES (SPARK)
    # =========================================================================
    print("\n   → Preparing product catalog and deriving rules from data...")
   
    product_cols = [c for c in df_products.columns]
    product_data = df_products.select(product_cols).collect()
   
    def safe_convert(value):
        if value is None:
            return None
        if isinstance(value, Decimal):
            return float(value)
        if isinstance(value, (int, float, str, bool)):
            return value
        return str(value)
   
    product_catalog = []
    for row in product_data:
        product_info = {'name': row.Product_Name}
        for col in product_cols:
            if col != 'Product_Name' and hasattr(row, col):
                value = getattr(row, col)
                converted_value = safe_convert(value)
                if converted_value is not None:
                    product_info[col.lower()] = converted_value
        product_catalog.append(product_info)
   
    product_list_parts = []
    for i, prod in enumerate(product_catalog):
        parts = [f"{i+1}. {prod['name']}"]
        if 'product_category' in prod:
            parts.append(f"Category: {prod['product_category']}")
        if 'target_audience' in prod:
            parts.append(f"Target: {prod['target_audience']}")
        if 'age_range' in prod:
            parts.append(f"Age Range: {prod['age_range']}")
        if 'minimum_balance' in prod:
            try:
                min_bal = float(prod['minimum_balance'])
                parts.append(f"Min Balance: ₦{min_bal:,.0f}")
            except (ValueError, TypeError):
                parts.append(f"Min Balance: {prod['minimum_balance']}")
        if 'interest_rate' in prod:
            parts.append(f"Interest Rate: {prod['interest_rate']}")
        for key, value in prod.items():
            if key not in ['name', 'product_category', 'target_audience', 'age_range', 'minimum_balance', 'interest_rate']:
                parts.append(f"{key.replace('_', ' ').title()}: {value}")
        product_list_parts.append(" - ".join(parts))
   
    product_list = "\n".join(product_list_parts)
   
    # =========================================================================
    # 5. BUILD DYNAMIC RULES FROM PRODUCT CATALOG
    # =========================================================================
    print("   → Deriving business rules from product catalog...")
   
    derived_rules = []
    for prod in product_catalog:
        prod_name = prod['name']
        if 'age_range' in prod and prod['age_range']:
            age_range = str(prod['age_range'])
            derived_rules.append(f"- {prod_name}: Only for customers within age range {age_range}")
        if 'target_audience' in prod and prod['target_audience']:
            target = prod['target_audience']
            derived_rules.append(f"- {prod_name}: Designed for {target}")
        if 'minimum_balance' in prod and prod['minimum_balance']:
            try:
                min_bal = float(prod['minimum_balance'])
                if min_bal > 0:
                    derived_rules.append(f"- {prod_name}: Requires minimum balance of ₦{min_bal:,.0f}")
            except (ValueError, TypeError):
                pass
   
    if additional_rules:
        derived_rules.extend([f"- {rule}" for rule in additional_rules])
   
    general_rules = [
        "- Match products to customer demographics (age, income, occupation)",
        "- Do NOT recommend products similar to customer's current account",
        "- Consider customer's financial capacity when recommending products",
        "- Respect all age ranges and target audience specifications STRICTLY"
    ]
   
    all_rules = general_rules + derived_rules
    rules_text = "\n".join(all_rules) if all_rules else "- Match products appropriately to customer profiles"
   
    print(f"      Generated {len(derived_rules)} product-specific rules from catalog")
   
    # =========================================================================
    # 6. USE OPENAI TO SCORE CUSTOMER-PRODUCT FIT
    # =========================================================================
    print("\n   → Using OpenAI to score product fit (intelligent matching)...")
    print("      This may take time depending on sample size...")
   
    customer_profiles_list = customer_profiles_sample.collect()
    total_batches = (len(customer_profiles_list) + batch_size - 1) // batch_size
    all_interactions = []
   
    for batch_idx in range(0, len(customer_profiles_list), batch_size):
        batch = customer_profiles_list[batch_idx:batch_idx + batch_size]
        print(f"      Processing batch {batch_idx//batch_size + 1}/{total_batches}...", end='')
       
        customer_summaries = []
        for cust in batch:
            summary_parts = [f"Customer {cust.Customer_ID}:"]
            for col in df_custs.columns:
                if col != 'Customer_ID' and hasattr(cust, col):
                    value = getattr(cust, col)
                    converted_value = safe_convert(value)
                    if converted_value is not None:
                        summary_parts.append(f"- {col.replace('_', ' ').title()}: {converted_value}")
            customer_summaries.append('\n'.join(summary_parts))
       
        prompt = f"""You are a banking product recommendation expert. Score how well each product fits each customer on a scale of 0-10.
 
# CUSTOMERS
{chr(10).join(customer_summaries)}
 
# PRODUCTS
{product_list}
 
# TASK
For each customer, score ONLY the top {top_n_products} most relevant products (0-10 scale).
 
# MATCHING RULES
{rules_text}
 
Return ONLY valid JSON with this exact structure:
{{
  "matches": [
    {{"customer_id": "C001", "product": "Exact Product Name", "score": 8}},
    {{"customer_id": "C001", "product": "Another Product", "score": 7}}
  ]
}}
 
Do NOT include any other text, only the JSON object."""
       
        try:
            response = client.chat.completions.create(
                model=model_name,
                messages=[
                    {"role": "system", "content": "You are a banking product expert. Return only valid JSON."},
                    {"role": "user", "content": prompt}
                ],
                temperature=temperature,
                response_format={"type": "json_object"}
            )
           
            response_text = response.choices[0].message.content.strip()
            if response_text.startswith('```'):
                response_text = response_text.split('```')[1]
                if response_text.startswith('json'):
                    response_text = response_text[4:]
                response_text = response_text.strip()
           
            result = json.loads(response_text)
            for match in result.get('matches', []):
                all_interactions.append({
                    'Customer_ID': match['customer_id'],
                    'Product_Name': match['product'],
                    'interaction_score': float(match['score'])
                })
           
            print(f" ✓ ({len(result.get('matches', []))} matches)")
        except Exception as e:
            print(f" ✗ Error: {e}")
            continue
       
        if batch_idx + batch_size < len(customer_profiles_list):
            time.sleep(rate_limit_delay)
   
    # =========================================================================
    # 7. CREATE TRANSACTION-BASED INTERACTIONS (WITH DESCRIPTION ANALYSIS)
    # =========================================================================
    interaction_matrix = None
   
    if use_transaction_data and df_trans is not None:
        print("\n   → Computing transaction-based interaction scores...")
        print("      Analyzing transaction descriptions for product signals...")
       
        # Get sampled customer IDs
        sampled_customer_ids = [row.Customer_ID for row in customer_profiles_sample.select('Customer_ID').collect()]
        df_trans_filtered = df_trans.filter(F.col('Customer_ID').isin(sampled_customer_ids))
       
        trans_filtered_count = df_trans_filtered.count()
        print(f"      Filtered to {trans_filtered_count:,} transactions for sampled customers")
       
        # Extract keywords from Account_Type
        df_trans_with_keywords = df_trans_filtered.withColumn(
            'account_keywords',
            F.regexp_extract(F.lower(F.col('Account_Type')),
                            r'(current|savings|sme|student|premium|platinum|children|aspire)', 1)
        )
       
        # Extract keywords from Description (if it exists)
        if 'Description' in df_trans_filtered.columns:
            print("      ✓ Description field found - using for enhanced matching")
            df_trans_with_keywords = df_trans_with_keywords.withColumn(
                'desc_keywords',
                F.regexp_extract(F.lower(F.col('Description')),
                                r'(current|savings|sme|student|premium|platinum|children|aspire|loan|investment|card|transfer|deposit)', 1)
            )
           
            # Combine keywords from both sources
            df_trans_with_keywords = df_trans_with_keywords.withColumn(
                'combined_keywords',
                F.when(F.col('account_keywords') != '', F.col('account_keywords'))
                 .when(F.col('desc_keywords') != '', F.col('desc_keywords'))
                 .otherwise('')
            )
        else:
            print("      ⚠ Description field not found - using Account_Type only")
            df_trans_with_keywords = df_trans_with_keywords.withColumn(
                'combined_keywords',
                F.col('account_keywords')
            )
       
        # Match transactions to products using combined keywords
        transaction_product_matches = df_trans_with_keywords.join(
            keyword_map.withColumnRenamed('product_keywords', 'combined_keywords'),
            'combined_keywords',
            'inner'
        )
       
        matches_count = transaction_product_matches.count()
        print(f"      Matched {matches_count:,} transactions to products")
       
        # Aggregate transaction metrics (RFM Analysis)
        transaction_interactions = transaction_product_matches.groupBy('Customer_ID', 'Product_Name').agg(
            F.count('*').alias('transaction_count'),
            F.sum('Trans_Amount').alias('total_amount'),
            F.max('Date').alias('last_transaction_date'),
            F.avg('Trans_Amount').alias('avg_amount')
        )
       
        # Calculate recency
        max_date = df_trans_filtered.agg(F.max('Date')).collect()[0][0]
       
        transaction_interactions = transaction_interactions.withColumn(
            'days_since_last',
            F.datediff(F.lit(max_date), F.col('last_transaction_date'))
        )
       
        # Create RFM-based interaction score (0-10 scale)
        print("      Computing RFM (Recency, Frequency, Monetary) scores...")
       
        # Frequency score: normalize transaction count (log scale for better distribution)
        transaction_interactions = transaction_interactions.withColumn(
            'frequency_score',
            F.least(F.lit(10.0), (F.log1p(F.col('transaction_count')) / F.log1p(F.lit(100.0))) * 10.0)
        )
       
        # Monetary score: normalize total amount (log scale)
        transaction_interactions = transaction_interactions.withColumn(
            'monetary_score',
            F.least(F.lit(10.0), (F.log1p(F.col('total_amount')) / F.log1p(F.lit(1000000.0))) * 10.0)
        )
       
        # Recency score: decay based on days since last transaction
        transaction_interactions = transaction_interactions.withColumn(
            'recency_score',
            F.when(F.col('days_since_last') <= 30, F.lit(10.0))
             .when(F.col('days_since_last') <= 60, F.lit(8.0))
             .when(F.col('days_since_last') <= 90, F.lit(6.0))
             .when(F.col('days_since_last') <= 180, F.lit(4.0))
             .when(F.col('days_since_last') <= 365, F.lit(2.0))
             .otherwise(F.lit(0.5))
        )
       
        # Combined RFM score with weights
        transaction_interactions = transaction_interactions.withColumn(
            'transaction_score',
            (F.col('frequency_score') * 0.35) +   # 35% frequency
            (F.col('monetary_score') * 0.35) +    # 35% monetary
            (F.col('recency_score') * 0.30)       # 30% recency
        )
       
        transaction_interactions = transaction_interactions.select(
            'Customer_ID',
            'Product_Name',
            F.col('transaction_score').alias('trans_score'),
            'transaction_count',
            'total_amount',
            'days_since_last'
        )
       
        trans_count = transaction_interactions.count()
        print(f"      Generated {trans_count:,} transaction-based interactions")
       
        # Show sample RFM scores
        print("\n      Sample RFM scores:")
        transaction_interactions.select(
            'Customer_ID', 'Product_Name', 'trans_score',
            'transaction_count', 'total_amount', 'days_since_last'
        ).show(5, truncate=False)
       
        # =========================================================================
        # 8. MERGE TRANSACTION AND OPENAI SCORES
        # =========================================================================
        print("\n   → Merging transaction and OpenAI scores...")
       
        if not all_interactions:
            print("      No OpenAI interactions, using transaction data only")
            interaction_matrix = transaction_interactions.select(
                'Customer_ID',
                'Product_Name',
                F.col('trans_score').alias('interaction_score')
            )
        else:
            openai_interactions = spark.createDataFrame(all_interactions)
           
            # Full outer join
            combined = openai_interactions.join(
                transaction_interactions.select('Customer_ID', 'Product_Name', 'trans_score'),
                ['Customer_ID', 'Product_Name'],
                'full_outer'
            )
           
            # Combine scores with weighting
            combined = combined.withColumn(
                'openai_score',
                F.coalesce(F.col('interaction_score'), F.lit(0.0))
            ).withColumn(
                'trans_score_filled',
                F.coalesce(F.col('trans_score'), F.lit(0.0))
            ).withColumn(
                'interaction_score',
                (F.col('trans_score_filled') * transaction_weight) +
                (F.col('openai_score') * (1.0 - transaction_weight))
            )
           
            interaction_matrix = combined.select(
                'Customer_ID',
                'Product_Name',
                'interaction_score'
            )
           
            combined_count = interaction_matrix.count()
            print(f"      Combined into {combined_count:,} total interactions")
            print(f"      Weighting: {transaction_weight*100:.0f}% transactions + {(1-transaction_weight)*100:.0f}% OpenAI")
   
    else:
        print("\n   → Using OpenAI scores only (no transaction data)")
        if all_interactions:
            interaction_matrix = spark.createDataFrame(all_interactions)
        else:
            print("   ⚠️ No interactions generated. Creating fallback...")
            fallback_count = min(3, len(all_product_names))
            fallback_products = all_product_names[:fallback_count]
            fallback_data = []
            for cust_row in customer_profiles_sample.select('Customer_ID').collect():
                for prod in fallback_products:
                    fallback_data.append((cust_row.Customer_ID, prod, 5.0))
            interaction_matrix = spark.createDataFrame(
                fallback_data,
                ['Customer_ID', 'Product_Name', 'interaction_score']
            )
   
    # =========================================================================
    # 9. AGGREGATE AND FINALIZE
    # =========================================================================
    print("\n   → Building final interaction matrix...")
   
    interaction_matrix = interaction_matrix.groupBy('Customer_ID', 'Product_Name').agg(
        F.max('interaction_score').alias('interaction_score')
    )
   
    # =========================================================================
    # 10. FILTER OUT CURRENT PRODUCTS
    # =========================================================================
    print("\n   → Filtering current products...")
   
    current_pairs = []
    for cust, prods in customer_current_products.items():
        for prod in prods:
            current_pairs.append((cust, prod))
   
    if current_pairs:
        current_pairs_df = spark.createDataFrame(
            current_pairs,
            ['Customer_ID', 'Product_Name']
        )
        interaction_matrix = interaction_matrix.join(
            current_pairs_df,
            ['Customer_ID', 'Product_Name'],
            'left_anti'
        )
   
    elapsed = time.time() - start_time
   
    # =========================================================================
    # FINAL STATS
    # =========================================================================
    total_interactions = interaction_matrix.count()
    unique_customers = interaction_matrix.select('Customer_ID').distinct().count()
    unique_products = interaction_matrix.select('Product_Name').distinct().count()
   
    print("\n" + "="*80)
    print("✅ HYBRID INTERACTION MATRIX CREATED")
    print("="*80)
    print(f"Total interactions: {total_interactions:,}")
    print(f"Unique customers: {unique_customers:,}")
    print(f"Unique products: {unique_products:,}")
    print(f"Products in catalog: {len(product_catalog)}")
    print(f"\n🧠 {model_name} + Transaction Data:")
    print(f"   • {len(derived_rules)} rules derived from product catalog")
    print(f"   • {len(additional_rules) if additional_rules else 0} custom business rules")
    print(f"   • Customer demographics and attributes")
    if use_transaction_data:
        print(f"   • Real transaction history with descriptions (weight: {transaction_weight*100:.0f}%)")
        print(f"   • RFM analysis: Recency (30%) + Frequency (35%) + Monetary (35%)")
    print(f"\n⚠️ Current products EXCLUDED from recommendations")
    print(f"   Customers with current products: {len(customer_current_products):,}")
    print(f"\n⚡ Completed in {elapsed:.1f} seconds")
    print(f"   Model used: {model_name}")
    print("="*80)
   
    return interaction_matrix, df_products, customer_current_products

print("✅ Function defined successfully")

✅ Function defined successfully


In [ ]:
# ============================================================================
# EXECUTE: Hybrid Interaction Matrix Generation
# ============================================================================

print("\n🚀 Starting Hybrid interaction matrix creation...\n")

custom_rules = [
    "Premium products require demonstrated high transaction volumes",
    "Savings products are suitable for customers with stable income"
]

interaction_df, product_map, customer_current_products = create_customer_product_interactions(
    df_customers,
    df_products,
    df_transactions,                # Transaction data with descriptions
    openai_api_key=openai_api_key,
    model_name=CONFIG['llm']['model'],
    customer_sample_size=1000,
    batch_size=30,
    temperature=CONFIG['llm']['temperature'],
    top_n_products=CONFIG['recommendation']['top_n'],
    rate_limit_delay=0.5,
    additional_rules=custom_rules,
    use_transaction_data=True,      # Enable transaction data
    transaction_weight=0.7           # 70% transactions, 30% OpenAI
)

print("\n📊 Sample Interactions:")
interaction_df.show(10)

print("\n🎯 Score Distribution:")
interaction_df.describe(['interaction_score']).show()

print("\n📈 Top Products by Interaction Count:")
top_products = interaction_df.groupBy('Product_Name').count().orderBy(F.desc('count')).limit(10)
top_products.show()


🚀 Starting Hybrid interaction matrix creation...

🔨 Creating interaction matrix (Hybrid: OpenAI + Transaction Descriptions)...

   Products in catalog: 42

   → Identifying current products from Account_Type...
   Products in catalog: 42

   → Identifying current products from Account_Type...
      Found current products for 92,202 customers
      Found current products for 92,202 customers

   → Preparing customer profiles...

   → Preparing customer profiles...
      Total customers: 100,000
      Sampling 1,000 customers from 100,000 total

   → Preparing product catalog and deriving rules from data...
   → Deriving business rules from product catalog...
      Generated 86 product-specific rules from catalog

   → Using OpenAI to score product fit (intelligent matching)...
      This may take time depending on sample size...
      Total customers: 100,000
      Sampling 1,000 customers from 100,000 total

   → Preparing product catalog and deriving rules from data...
   → Deriving 